In [1]:
!pip install tensorflow

In [2]:
!pip install numpy

In [3]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 863.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 

In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img , ImageDataGenerator
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense , Dropout , Flatten , Conv2D , MaxPooling2D


In [30]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shreyapmaher/fruits-dataset-images")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/shreyapmaher/fruits-dataset-images/versions/1


In [31]:
dirs = os.listdir('/kaggle/input/fruits-dataset-images/images')
for dir in dirs:
  files = list(os.listdir('/kaggle/input/fruits-dataset-images/images/'+dir))
  print(dir + ' Folder has: ' + str(len(files))+'images')


apple fruit Folder has: 40images
orange fruit Folder has: 40images
strawberry fruit Folder has: 40images
grapes fruit Folder has: 40images
banana fruit Folder has: 40images
cherry fruit Folder has: 40images
kiwi fruit Folder has: 40images
mango fruit Folder has: 40images
chickoo fruit Folder has: 40images


In [32]:
base_dir = '/kaggle/input/fruits-dataset-images/images'
img_size = 180
batch = 32

In [33]:
train_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

val_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

Found 360 files belonging to 9 classes.
Using 288 files for training.
Found 360 files belonging to 9 classes.
Using 72 files for validation.


In [34]:
fruit_names = train_ds.class_names
fruit_names

['apple fruit',
 'banana fruit',
 'cherry fruit',
 'chickoo fruit',
 'grapes fruit',
 'kiwi fruit',
 'mango fruit',
 'orange fruit',
 'strawberry fruit']

In [35]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [36]:
data_augmentation = Sequential([
    layers.RandomFlip('horizontal',input_shape=(img_size,img_size,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10)
])

In [38]:
# ... (previous code) ...

# Assuming there are 120 breeds in the dataset
num_classes = 120

model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes) # Changed to num_classes
])

# ... (rest of the code) ...

In [39]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 120)            │        15,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,004,120 (15.27 MB)

 Trainable params: 4,004,120 (15.27 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
history = model.fit(train_ds,validation_data=val_ds,epochs=15)

Epoch 1/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.0734 - loss: 4.0163 - val_accuracy: 0.1528 - val_loss: 2.2353
Epoch 2/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.2060 - loss: 2.1745 - val_accuracy: 0.0833 - val_loss: 2.1077
Epoch 3/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.1936 - loss: 2.0349 - val_accuracy: 0.3472 - val_loss: 1.7592
Epoch 4/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4032 - loss: 1.6826 - val_accuracy: 0.4028 - val_loss: 1.7047
Epoch 5/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4791 - loss: 1.4515 - val_accuracy: 0.3611 - val_loss: 1.6297
Epoch 6/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.4863 - loss: 1.3944 - val_accuracy: 0.4306 - val_loss: 1.6383
Epoch 7/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5810 - loss: 1.2743 - val_accuracy: 0.4306 - val_loss: 1.6849
Epoch 8/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5930 - loss: 1.1816 - val_accuracy: 0.5000 - val_loss: 1.4937
Epoch 9/

In [42]:
def classify_image(image_path):
  input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
  input_image_array = tf.keras.utils.img_to_array(input_image)
  input_image_exp_dim = tf.expand_dims(input_image_array,0)
  predictions = model.predict(input_image_exp_dim)
  result = tf.nn.softmax(predictions[0])
  predicted_class_index = np.argmax(result)

  # Check if index is within bounds of fruit_names
  if predicted_class_index < len(fruit_names):
    outcome= 'the fruit belong to '+ fruit_names[predicted_class_index] + ' with a score of '+str(max(result)*100)
  else:
    outcome = f"Predicted class index ({predicted_class_index}) is out of range for fruit_names (length: {len(fruit_names)}). This indicates a mismatch between the model's output and the available class labels."

  return outcome

In [43]:
classify_image('/content/Kiwi-fruit.webp')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


'the fruit belong to kiwi fruit with a score of tf.Tensor(74.07407, shape=(), dtype=float32)'